In [51]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

## Credit Card Fraud Detection

고객이 구매하지 않은 항목에 대해 비용이 청구되지 않도록 신용 카드 회사가 사기 신용 카드 거래를 인식할 수 있는 것이 중요합니다.

콘텐츠

데이터 세트에는 2013년 9월 유럽 카드 소지자가 신용 카드로 수행한 거래가 포함되어 있습니다.
이 데이터 세트는 284,807건의 거래 중 492건의 사기가 있는 2일 동안 발생한 거래를 나타냅니다. 데이터 세트는 매우 불균형하며 긍정적인 클래스(사기)가 모든 거래의 0.172%를 차지합니다.

여기에는 PCA 변환의 결과인 숫자 입력 변수만 포함됩니다. 유감스럽게도 기밀 문제로 인해 원본 기능과 데이터에 대한 자세한 배경 정보를 제공할 수 없습니다. 피처 V1, V2, … V28은 PCA로 얻은 주요 구성 요소이며 PCA로 변환되지 않은 유일한 특징은 '시간'과 '양'입니다. 피처 '시간'에는 각 트랜잭션과 데이터 세트의 첫 번째 트랜잭션 사이에 경과된 초가 포함됩니다. 'Amount' 기능은 거래 금액이며, 이 기능은  비용에 민감한 학습(example-dependant cost-sensitive learning)에 사용할 수 있습니다. Feature 'Class'는 응답 변수이며 사기의 경우 값 1, 그렇지 않은 경우 0입니다.

주어진 데이터셋으로 아래와 같이 모델링 및 평가를 수행하세요.
- 로지스틱 회귀와  LightGBM 모델을 사용하여 모델 학습, 예측, 평가
- 중요한 변수로 Amount 피처를 표준화한 후 모델 학습, 예측, 평가
- Amount 피처를 로그 변환한 후 모델 학습, 예측, 평가
- 이상치 데이터를 처리한 후 모델 학습, 예측, 평가
- SMOTE 오버 샘플링 적용 후 모델 학습, 예측, 평가

In [10]:
import pandas as pd
import numpy as np

card_df = pd.read_csv('./creditcard.csv')
card_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [12]:
from sklearn.model_selection import train_test_split

def get_preprocessed_df(df=None):
    df_copy = df.copy()
    df_copy.drop('Time',axis=1,inplace=True)
    return df_copy

In [15]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수

def get_train_test_dataset(df=None):
    df_copy = get_preprocessed_df(df)
    
    X_features = df_copy.iloc[:,:-1]
    y_target = df_copy.iloc[:,-1]
    
    X_train,X_test,y_train,y_test = train_test_split(X_features,y_target,test_size=0.3, random_state=123, stratify = y_target)
    
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test = get_train_test_dataset(card_df)

In [17]:
y_train.value_counts()

0    199020
1       344
Name: Class, dtype: int64

In [22]:
y_train.count()

199364

In [25]:
y_train.shape[0]

199364

In [26]:
train_cnt = y_train.count()
test_cnt = y_test.count()

print(y_train.value_counts()/train_cnt*100) # -> 학습 세트 레이블 값 분포 비율
print(y_train.value_counts()/test_cnt*100) # -> 테스트 세트 레이블 값 분포 비율

0    99.827451
1     0.172549
Name: Class, dtype: float64
0    232.927215
1      0.402608
Name: Class, dtype: float64


In [27]:
print(y_train.value_counts()/y_train.shape[0]*100)
print(y_test.value_counts()/y_test.shape[0]*100)

0    99.827451
1     0.172549
Name: Class, dtype: float64
0    99.826785
1     0.173215
Name: Class, dtype: float64


In [ ]:
def get_clf_

In [33]:
# light GBM 사용해서 객체 만들기
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from lightgbm import LGBMClassifier


X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train, test_size=0.2,random_state=123)

lgbm_wrapper = LGBMClassifier(n_estimator=200, learning_rate=0.05)
evals = [(X_tr,y_tr),(X_val,y_val)]
lgbm_wrapper.fit(X_tr,y_tr,early_stopping_rounds=50, eval_metric='logloss',eval_set=evals,verbose=True)
preds = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)[:,-1]

[LightGBM] [Warning] Unknown parameter: n_estimator
[1]	training's binary_logloss: 0.00588554	valid_1's binary_logloss: 0.005901
[2]	training's binary_logloss: 0.0115789	valid_1's binary_logloss: 0.0162172
[3]	training's binary_logloss: 0.00451308	valid_1's binary_logloss: 0.00662061
[4]	training's binary_logloss: 0.00415713	valid_1's binary_logloss: 0.00674977
[5]	training's binary_logloss: 0.00390251	valid_1's binary_logloss: 0.00669884
[6]	training's binary_logloss: 0.00365537	valid_1's binary_logloss: 0.00666497
[7]	training's binary_logloss: 0.00342644	valid_1's binary_logloss: 0.00660417
[8]	training's binary_logloss: 0.00328798	valid_1's binary_logloss: 0.00651837
[9]	training's binary_logloss: 0.00316933	valid_1's binary_logloss: 0.00645037
[10]	training's binary_logloss: 0.00298789	valid_1's binary_logloss: 0.00639679
[11]	training's binary_logloss: 0.0028585	valid_1's binary_logloss: 0.00631404
[12]	training's binary_logloss: 0.00272959	valid_1's binary_logloss: 0.00623607
[1

In [37]:
from hyperopt import hp

lgbm_search_space = {'max_depth':hp.quniform('max_depth',5,20,1),
                    'min_child_weight': hp.quniform('min_child_weight',1,2,1),
                    'colsample_bytree' : hp.uniform('colsample_bytree',0.5,1),
                    'learning_rate':hp.uniform('learning_rate',0.5,1)}

In [ ]:
# 멀티로 모델을 만들 때 모델을 인자값으로 주고, 여러 인자 값 집어넣으면 됨!

def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train)
    

In [ ]:
# n_jobs = -1 : 학습 및 예측에 사용할 코어 수를 최대로 설정
# boost_from_average = False : 클래스 불균형 문제를 다루기 위한 LGBM의 특정 기능을 비활성화



In [ ]:
# 그래프 확인

In [ ]:
# 정규분포 형태로 변환

In [ ]:
from sklearn.preprocessing import StandardScaler

def get_proprocessed_df(df = None):
    df_copy = df.copy()
    scaler = StandardScaler()
    amount_n = scaler.fit_transform(df_copy['Amount'].values.reshape(-1,1))
    df_copy.insert(0,'Amount_Scaled',amount_n)
    #df_copy.insert(lic, column, value, allow_duplicates=False -> 중복허락 True)
    df_copy.drop(['Amount','Time'],axis=1,inplace=True)
    return df_copy

** insert 함수 -> 칼럼 추가 함수, 0번째 인덱스에 설정한 이름으로 이름 짓고 내용을 값으로 설정

이후 기존 amount 삭제, time도 필요없어서 삭제


In [ ]:
def get_train_test_dataset(card_df)

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)

print('##로지스틱 회귀 예측 성능##')
lr_clf = LogisticRegression()
get_model_train_eval(lr_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test = y_test)

print('##light GBM 회귀 예측 성능##')
수정~~lr_clf = LogisticRegression()
get_model_train_eval(lr_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test = y_test)

#### 로그변환

In [ ]:
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    amount_n = np.log1p(df_copy['Amount'])
    df_copy.insert(0,'Amount_Scaled',amount_n)
    df_copy.drop(['Time','Amount'],axis=1,inplace=True)
    return df_copy
    
    
# log1p : 1에 1을 더한 후 로그값을 계산함. 이유 - 입력값이 0 또는 음수인 경우에도 정의된 결과를 얻기 위함임.

#### 이상치 데이터 제거 후 모델 학습/예측/평가
- 이상치로 인해 머신러닝 모델의 성능에 영향을 받을 경우가 쉬움
- 적기

In [ ]:
import seaborn as sns

plt.figure(figsize=(9,9))
corr = card_df.corr()
sns.heatmap(corr, cmap='RaBu')

In [ ]:
import numpy as np

def get_outlier(df=None, column=None, weight=1.5):
    fraud = df[df['Class']==1][column]
    quantile_25 = np.percentile(frayd.values,25)
    quantile_75 = np.percentile(frayd.values,75)
    
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr*weight
    lowest_val = quantile_25 - iqr_weight
    highset_val = quantile_75 + iqr_weight
    
    outlier_index = fraud[(fraud > highset_val) | (fraud < lowest_val)].index
    return outlier_index

In [ ]:
get_preprocessed_df에 이상치 제거까지 추가하기

### SMOTE 오버 샘플링 적용 후 모델 학습/예측/평가
- imbalanced-learn 패키지의 SMOTE 클래스를 이용해 오버 샘플링 적용
- 반드시 학습 데이터 세트만 오버 샘플링 적용
- SMOTE를 적용하면 재현율은 높아지나 정밀도는 낮아지는 것이 일반적
- 좋은 SMOTE 패키지일수록 재현율 증가율은 높이고 정밀도 감소율은 낮출 수 있도록 효과적으로 데이터 증식

In [38]:
import sklearn
print(sklearn.__version__)

1.0.2


In [39]:
!pip install imbalanced-learn

     ------------------------------------- 226.0/226.0 kB 13.5 MB/s eta 0:00:00
     ------------------------------------- 298.0/298.0 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


In [40]:
from imblearn.over_sampling import SMOTE

In [44]:
smote = SMOTE(random_state=0)
X_train_over, y_train_over = smote.fit_resample(X_train,y_train)
print('smote 적용 전 피처/레이블 세트: ',X_train.shape, y_train.shape)
print('smote 적용 후 피처/레이블 세트: ',X_train_over.shape, y_train_over.shape)
print('smote 적용 후 레이블 값 분포:\n',pd.Series(y_train_over).value_counts())

smote 적용 전 피처/레이블 세트:  (199364, 29) (199364,)
smote 적용 후 피처/레이블 세트:  (398040, 29) (398040,)
smote 적용 후 레이블 값 분포:
 0    199020
1    199020
Name: Class, dtype: int64


In [ ]:
# lr 모델이 오버샘플링으로 인해 실제 원본 데이터의 유형보다 너무나 많은 class=1 데이터를 학습하면서
# 실제 테스트 세트에서 예측을 지나치게 class=1로 적용해 정밀도가 급격하게 하락

from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
get_model_train_eval(lr_clf, ftr_train=X_train_over, ftr_test=X_test, tgt_train=y_train_over, tgt_test=y_test)

In [46]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.metrics import precision_recall_curve
%matplotlib inline

def precision_recall_curve_plot(y_test , pred_proba_c1):
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출. 
    precisions, recalls, thresholds = precision_recall_curve( y_test, pred_proba_c1)
    
    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행. 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary],label='recall')
    
    # threshold 값 X 축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    
    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
    plt.legend(); plt.grid()
    plt.show()

In [ ]:
precision_recall_curve_plot(y_test, lr_clf.predict_proba)